In [3]:
import requests
import json
url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama3.1:8b",   
    "prompt": "Hi"
}

response = requests.post(url, json=payload, stream=True)

# print(response.text)
output_text = ""
for line in response.iter_lines():
    if line:
        data = json.loads(line.decode("utf-8"))
        if "response" in data:
            output_text += data["response"]
        if data.get("done", False): 
            break

print("Final Output:\n", output_text)

Final Output:
 It's nice to meet you. Is there something I can help you with or would you like to chat?


In [4]:
from ollama import Client

In [3]:
# will work on vectore store database
import pandas as pd

data=pd.read_csv('./data/realistic_restaurant_reviews.csv')

In [4]:
import os
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings



In [2]:
def store_in_chroma(df):
    if "Review" not in df.columns:
        raise ValueError("DataFrame must contain a 'Review' column")
    
    texts = df["Review"].dropna().astype(str).tolist()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    documents = text_splitter.create_documents(texts)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print('EMbdi',embeddings)
    persist_dir = "./chroma_db_sent"
    os.makedirs(persist_dir, exist_ok=True)
    vectorstore = Chroma(
        collection_name="collection",
        embedding_function=embeddings,
        persist_directory=persist_dir
    )

    existing_docs = vectorstore.get()
    existing_texts = set(existing_docs["documents"]) if existing_docs else set()
    prev_count = len(existing_texts)
    new_texts = [doc.page_content for doc in documents if doc.page_content not in existing_texts]
    if new_texts:
        vectorstore.add_texts(new_texts)
        vectorstore.persist()
        print(f"Added {len(new_texts)} new chunks to ChromaDB")
    else:
        print("No new data to add, everything already exists in ChromaDB")

    updated_docs = vectorstore.get()
    total_count = len(updated_docs["documents"]) if updated_docs else 0
    

    
    
    
    return vectorstore


In [ ]:
vect_data=store_in_chroma(data)
print(vect_data)


NameError: name 'data' is not defined

In [8]:
print(vect_data)

In [15]:
def get_retriever():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma(
        collection_name="collection",   
        persist_directory="./chroma_db_sent",
        embedding_function=embeddings
    )
    retriever = vectorstore.as_retriever(
        search_type="similarity",   
        search_kwargs={"k": 3}    
    )

    return retriever

In [16]:
retriever = get_retriever()
docs = retriever.get_relevant_documents("Disappointed with service ?")
print(docs)

[Document(page_content='90-minute wait on a Thursday night with no option to call ahead. The pizza was good when we finally got it, but nothing special enough to justify such a long wait. Many comparable options with better systems.'), Document(page_content="Nothing terrible but nothing special either. The crust was okay, toppings were standard, and service was fine. It's the kind of place you go when you're in the area, but wouldn't make a special trip for."), Document(page_content="Ordered for Valentine's Day and regretted it. Pizza took 2 hours to arrive, was cold, and had the wrong toppings. Called the restaurant and they seemed completely unconcerned. Never again.")]


In [17]:
print(data.head())   # should be > 0


                          Title        Date  Rating  \
0            Best pizza in town  2024-03-15       5   
1     Disappointed with service  2024-02-20       2   
2  Authentic Italian experience  2024-01-05       5   
3   Overpriced for what you get  2024-03-01       3   
4      Great gluten-free option  2024-02-15       4   

                                              Review  
0  The crust was perfectly crispy on the outside ...  
1  While the pizza itself was decent, we waited o...  
2  This place reminds me of the pizzerias in Napl...  
3  $24 for a medium pizza with just two toppings ...  
4  As someone with celiac disease, finding good g...  


In [18]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.1:8b")

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

QA_TEMPLATE = """
You are an expert about answering questions about a pizza restaurant.
Always give clear, helpful, and concise answers.

Use the following pieces of retrieved context to answer the user’s question.
If you don’t know the answer,try to get it from your knowledge and append  this sentence ,"i'm not sure about this" ,don’t make things up.
And make sure answer must be in 2 to 3 lines.
Context:
{context}

Question: {question}

Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=QA_TEMPLATE,
)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever, 
    chain_type="stuff", #For converting context into prompt 
    chain_type_kwargs={"prompt": qa_prompt}
)


query = "Late night savior?"
result = qa_chain.run({"query": query})


C:\Users\ddurande\AppData\Local\Temp\ipykernel_10032\2708740973.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain.run({"query": query})


In [20]:
print(result)

This pizza place has been a late-night lifesaver, open until 3am on weekends. They've got huge slices that are reheated to perfection and creative special menu items that hit the spot when you're hungry late at night. Perfect for those long nights out!


In [ ]:
llm_fallback = Ollama(model="llama3.1:8b")

In [ ]:

from langchain_community.llms import Ollama
# this will work if the question is outsider or which is not releated to the document
def process_with_fallback(result: str, query: str) -> str:
    irrelevant_markers = [
        "doesn't seem to be about",
        "this conversation doesn't seem",
        "i'd be happy to try and help",
        "not related",
        "i'm not sure about this",
        "this question is not relevant",
        "out of context"
    ]
    if any(marker.lower() in result.lower() for marker in irrelevant_markers):
        # print("Irrelevant result detected, switching to fallback LLM...")
        prompt = f"""
            Please answer the following question in only 2–3 concise lines:

            Question: {query}
            """
        fallback_answer = llm_fallback.invoke(prompt)
        return fallback_answer
    
    return result

In [ ]:
print(process_with_fallback(result,query))

Yes, they fit the bill as a late-night savior with their creative and delicious menu items. Their special late-night options are perfect for hitting the spot when hunger strikes in the middle of the night.
